# DDI BDM
This notebook calculates the algoritmic complexity of the drug interaction network of the DECAGON dataset. The dataset is given as a list of adjacency matrices, each of dimension $N_{drugs} \times N_{drugs}$, corresponding to the connectivity per each joint side effect. The code uses the package pybdm to calculate the complexity contribution of each node and its corresponding edges per side effect. The result is a list of feature vectors,  exported as a `pickle` readable format file along with relevant data.

## Python 3

In [1]:
import numpy as np
import time
import os
import psutil
import pickle
import datetime as dt
from pybdm import BDM
from pybdm.partitions import PartitionRecursive
from algorithms import PerturbationExperiment, NodePerturbationExperiment
from getpass import getuser

In [2]:
# Settings and loading of the list of adj matrices
input_file = './data_structures/DS/DS_toy_genes500_drugs400_se3'
start = time.time() 
pid = os.getpid()
ps= psutil.Process(pid)
with open(input_file, 'rb') as f:
    ddi_adj_list = pickle.load(f)['ddi_adj_list']
print('Input data loaded')
jobs = 8
usrnm = getuser()
bdm = BDM(ndim=2, partition=PartitionRecursive)

Input data loaded


In [3]:
# DDI feature vector list
start1 = time.time()
nodebdm_ddi_list = []
add_edgebdm_ddi_list = []
rem_edgebdm_ddi_list = []
ddi_nodeper = NodePerturbationExperiment(bdm,metric='bdm',bipartite_network=False,
                                          parallel=True,jobs=jobs)
ddi_edgeper = PerturbationExperiment(bdm, bipartite_network=False)
total = len(ddi_adj_list)
count=1
for i in ddi_adj_list:
    ddi_nodeper.set_data(np.array(i.todense()))
    ddi_edgeper.set_data(np.array(i.todense()))
    print('set data')
    nodebdm_ddi_list.append(ddi_nodeper.run())
    add_edgebdm_ddi_list.append(ddi_edgeper.run_adding_edges())
    rem_edgebdm_ddi_list.append(ddi_edgeper.run_removing_edges())
    prog = count*100/total
    count += 1
    print(prog,'% completed')
print('Node and Edge BDM for DDI calculated')
print(dt.timedelta(seconds=time.time()-start1))

set data
33.333333333333336 % completed
set data


/home/juan/Documents/thesis/decagon-master/data/algorithms.py:318: RuntimeWarning: invalid value encountered in true_divide
  row_nodes = np.nan_to_num(row_sum/row_nz, nan=0.0)


66.66666666666667 % completed
set data
100.0 % completed
Node and Edge BDM for DDI calculated
0:03:19.663328


/home/juan/Documents/thesis/decagon-master/data/algorithms.py:318: RuntimeWarning: invalid value encountered in true_divide
  row_nodes = np.nan_to_num(row_sum/row_nz, nan=0.0)


In [4]:
# EXPORTING
drugs = np.shape(ddi_adj_list[0])[0]
memUse = ps.memory_info()
total_time = time.time()-start
output_data = {}
output_data['nodebdm_ddi_list'] = nodebdm_ddi_list
output_data['add_edgebdm_ddi_list'] = add_edgebdm_ddi_list
output_data['rem_edgebdm_ddi_list'] = rem_edgebdm_ddi_list
output_data['vms_ddi'] = memUse.vms
output_data['rss_ddi'] = memUse.rss
output_data['time_ddi'] = total_time
output_data['jobs_ddi'] = jobs

In [5]:
path = os.getcwd()
words = input_file.split('_')
print(words)

['./data', 'structures/DS/DS', 'toy', 'genes500', 'drugs400', 'se3']


In [6]:
output_file = path + '/data_structures/BDM/DDI_BDM_' + words[2] + '_se_' + str(total) +\
             '_drugs_' + str(drugs) + '_' + usrnm + str(jobs)
with open(output_file, 'wb') as f:
    pickle.dump(output_data, f, protocol=3)
print('Output data exported')

Output data exported
